In [31]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [32]:
base_treino = pd.read_csv(r"D:\pipeline_dados\data_raw\base_orcamentos_treino.csv")
x_teste_df = pd.read_csv(r"D:\pipeline_dados\base_orcamentos_nov.csv")

In [33]:
y = base_treino['convertido']


base_treino = base_treino.drop(['convertido', 'id_quote'], axis=1)
x_teste_df = x_teste_df.drop(['convertido', 'id_quote'], axis=1)


categoricos = ['ds_tipo_obra', 'nm_prop_oportunidade', 'proprietario', 'promocional', 'tipo_loja',
               'banheiro', 'toda_casa', 'cozinha', 'area_externa', 'itens_assentamento', 'sala',
               'piscina', 'area_servico', 'garagem', 'dormitorio', 'varanda', 'escada', 'amb_outros']

In [34]:

date_column = 'dt_criacao'

if date_column in base_treino.columns:
    print(f"Removing date column: {date_column}")
    base_treino = base_treino.drop(date_column, axis=1, errors='ignore')
    if date_column in categoricos:
        categoricos.remove(date_column)
else:
    print(f"Date column '{date_column}' not found in the training data.")

if date_column in x_teste_df.columns:
    x_teste_df = x_teste_df.drop(date_column, axis=1, errors='ignore')
    if date_column in categoricos:
        categoricos.remove(date_column)
else:
    print(f"Date column '{date_column}' not found in the test data.")

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

encoder = OneHotEncoder(drop='if_binary', handle_unknown='ignore')
X_encoded = pd.DataFrame(encoder.fit_transform(base_treino[categoricos]).toarray(),
                          columns=encoder.get_feature_names_out(categoricos))

X_final = pd.concat([base_treino.drop(categoricos, axis=1), X_encoded], axis=1)

X_test_encoded = pd.DataFrame(encoder.transform(x_teste_df[categoricos]).toarray(),
                               columns=encoder.get_feature_names_out(categoricos))
X_test_final = pd.concat([x_teste_df, X_test_encoded], axis=1).drop(categoricos, axis=1).fillna(0)

X = X_final.select_dtypes(exclude=['datetime64[ns]'])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)





Removing date column: dt_criacao


d:\spark_testes\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [35]:
xgb_model = XGBClassifier()


xgb_model.fit(X_train, y_train)


y_pred = xgb_model.predict(X_valid)

y_test_prob = xgb_model.predict_proba(X_test_final)
y_test_pred = xgb_model.predict(X_test_final)

result_df = pd.DataFrame({'predicted_class': y_test_pred, 'probabilidade_S': y_test_prob[:, 1], 'probabilidade_N': y_test_prob[:, 0]})

print(result_df)

     predicted_class  probabilidade_S  probabilidade_N
0                  0         0.469579         0.530421
1                  1         0.896387         0.103613
2                  1         0.981489         0.018511
3                  0         0.004417         0.995583
4                  1         0.906978         0.093022
..               ...              ...              ...
235                1         0.714420         0.285580
236                0         0.104772         0.895228
237                0         0.110570         0.889430
238                0         0.377477         0.622523
239                1         0.704969         0.295031

[240 rows x 3 columns]
